In [34]:
import scipy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date
import statistics as st
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.distributions.empirical_distribution import ECDF
from TimeSerie_fct import create_monthly_avg_time_serie
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import acf, pacf
import os


data_temperature = pd.read_table('../data/observatoire-geneve/TG_STAID000241.txt',sep = ',',
                                names = ['SOUID','DATE','TG','Q_TG'], skiprows = range(0,20))

data_temperature.drop(data_temperature[ data_temperature['Q_TG'] == 9 ].index, inplace = True)
data_temperature['Year'] = [int(str(d)[:4]) for d in data_temperature.DATE]
data_temperature['Month'] = [int(str(d)[4:6]) for d in data_temperature.DATE]
data_temperature['Day'] = [int(str(d)[6:8]) for d in data_temperature.DATE]

#Compute the day of the year for each year
day_of_year = np.array(len(data_temperature['Day']))

adate = [datetime.strptime(str(date),"%Y%m%d") for date in data_temperature.DATE]
data_temperature['Day_of_year'] = [d.timetuple().tm_yday for d in adate]
data_temperature.TG = data_temperature.TG/10.

In [35]:
df = data_temperature.copy()
plt.style.use("ggplot")

In [36]:
# Transformation en moyenne annuelle
Years = df.Year.unique()
data_Y = pd.DataFrame(np.array([[df[df.Year == y].TG.mean(),
                        df[df.Year == y].TG.median(),df[df.Year == y].TG.std(),y] for y in Years]),
                     index = (np.arange(np.shape(Years)[0])), columns=["Mean","Median","Std","Years"])
#Rupture de la moyenne en 1962

In [37]:
# QQ-plot of the annual mean
fig = qqplot(data_Y.Mean,loc = data_Y.Mean.mean(), scale = data_Y.Mean.std(), line ='45')
fig.suptitle("QQ-plot",y = 0.95,size='xx-large',weight = 'roman')
#plt.show()

plt.savefig("figure/Annual_qqplot.png", dpi=300, bbox_inches='tight')
plt.close(fig)

In [38]:
ecdf = ECDF(data_Y.Mean)
mean = data_Y.Mean.mean()
x = np.linspace(-4.+mean,4.+mean,5000)

fig, axs = plt.subplots(1,2, figsize=(14,4))

std = data_Y.Mean.std()
axs[0].plot(x,ecdf(x),"k",label='ecdf')
axs[0].plot(x,sc.stats.norm.cdf(x,loc = data_Y.Mean.mean(),scale = std), "b--", label = 'cdf')
axs[1].plot(x,np.abs(ecdf(x)-sc.stats.norm.cdf(x,loc = data_Y.Mean.mean() ,scale = std)),color = 'k')

axs[0].set(xlabel="x",ylabel="P (X<=x)")
axs[1].set(xlabel="x",ylabel="Absolute deviation")
axs[0].set_title("Ecdf vs cdf")
axs[1].set_title("Absolute deviation of the ecdf from the cdf")

axs[0].legend(loc = (0.05,0.8))

plt.savefig("figure/Annual_ecdf_cdf.png", dpi=300, bbox_inches='tight')
plt.close(fig)

In [157]:
fig, axs = plt.subplots(3, figsize=(10,8))

axs[0].plot(Years,data_Y.Mean, color = 'k')
axs[0].set_title("Annual mean temperature from 1901 to 2021")
axs[1].plot(Years,data_Y.Median, color = 'k')
axs[1].set_title("Annual median temperature from 1901 to 2021")
axs[2].plot(Years,data_Y.Std, color = 'k')
axs[2].set_title("Annual standard deviation of temperature from 1901 to 2021")
plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.6)
axs[0].set(xlabel="Years",ylabel="Temperature (°C)")
axs[1].set(xlabel="Years",ylabel="Temperature (°C)")
axs[2].set(xlabel="Years",ylabel="Standard deviation(°C)")



plt.savefig("figure/Annual_mean_median_std.png", dpi=300, bbox_inches='tight')
plt.close(fig)

In [42]:
#Calcul des anomalies normalisées

mean = data_Y.Mean.mean()
data_Y["anomalie_norm"] = (data_Y.Mean-mean)/(data_Y.Mean-mean).std()

mean1 = data_Y[data_Y.Years<=1962].Mean.mean()
mean2 = data_Y[data_Y.Years>1962].Mean.mean()
anom1 = (data_Y[data_Y.Years<=1962].Mean-mean1)/(data_Y[data_Y.Years<=1962].Mean-mean1).std()
anom2 = (data_Y[data_Y.Years>1962].Mean-mean2)/(data_Y[data_Y.Years>1962].Mean-mean2).std()

In [43]:
fig, axs = plt.subplots(3, figsize=(10,6))

fig.suptitle("Annual standardized temperature anomalies :",y = 1.1,size='x-large',weight = 'roman')

axs[0].bar(Years,data_Y.anomalie_norm, color='k')
axs[0].set_title("Over the period 1901-2021")
axs[1].bar(Years[Years<=1962],anom1, color = 'k')
axs[1].set_title("Over the period 1901-1962")
axs[2].bar(Years[Years>1962],anom2, color = 'k')
axs[2].set_title("Over the period 1963-2021")

axs[0].set(xlabel="Years")
axs[1].set(xlabel="Years")
axs[2].set(xlabel="Years")

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=1, 
                    wspace=0.4, 
                    hspace=0.6)

plt.savefig("figure/Annual_anomalie_temperature.png", dpi=300, bbox_inches='tight')
plt.close(fig)

In [44]:
#Stationarity test on annual mean with addfuller
test_addfuller = adfuller(data_Y.Mean, maxlag=None, regression='nc', autolag='AIC'
                          , store=False, regresults=False)

g = open('txt/addfullerTest_annual.txt','w')
g.write("Annual mean (no constant) : p-value = "+str(test_addfuller[1])+'\n')
g.close()

#Stationarity test on annual mean with addfuller
test_addfuller = adfuller(data_Y.Mean, maxlag=None, regression='ct', autolag='AIC'
                          , store=False, regresults=False)

g = open('txt/addfullerTest_annual.txt','a')
g.write("Annual mean (constant + trend) : p-value = "+str(test_addfuller[1])+'\n')
g.close()

In [98]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 300)

fig.suptitle("ACF and PACF of annual mean temperature",y = 1.1,size='x-large',weight = 'roman')

plot_acf(data_Y.Mean, lags=50,ax=axes[0], color='k',zero = False)
plot_pacf(data_Y.Mean, lags=50,ax=axes[1], color='k',zero = False)

axes[0].set(xlabel="lags")
axes[1].set(xlabel="lags")

plt.savefig("figure/Annual_acf_pacf.png", dpi=300, bbox_inches='tight')
plt.close(fig)

In [46]:
#Test U de Mann-Whitney
# Null hypothesis is that X and Y the probability of X being 
# greater than Y is equal to the probability of Y being greater than X.

p_values = np.array([])

for i in range(1,np.shape(data_Y.Mean)[0]-1):
    x = data_Y.Mean[:i]
    y = data_Y.Mean[i:]
    p = np.array([sc.stats.mannwhitneyu(x, y, use_continuity=True, alternative='two-sided').pvalue])
    # p = np.array([sc.stats.kendalltau(x, y).pvalue])
    p_values = np.concatenate([p_values,p])

alpha = 0.05

fig = plt.figure(figsize=(8,4), dpi=300, tight_layout = True)
fig.suptitle("P-values of the Mann-Whitney U test",y = 1.,size='x-large',weight = 'roman')

for i in range(1,np.shape(data_Y.Mean)[0]-1):
    if (p_values[i-1]>alpha):
        line1, = plt.plot(i,p_values[i-1],'o',color='red',label = 'p-value > '+str(alpha))
    else:
        line2, =plt.plot(i,p_values[i-1],'o',color='grey',label='p-value <= '+str(alpha))

        plt.axhline(y=0.05,linestyle='--',label='p_value = '+str(alpha))

fig.legend([line1,line2],['p-value > '+str(alpha),'p-value <= '+str(alpha)],loc = (0.775,0.73))
plt.xlabel('Size of \'left\' sample in the U test')
plt.ylabel('p-values')

plt.savefig("figure/p_values_MannWhitneyTest.png", dpi=300, bbox_inches='tight')
plt.close(fig)

# We can easily reject the null hypothesis at a level of 0.05 (or smaller)

In [47]:
#Test Q de Ljung-Box

test_Q = sm.stats.acorr_ljungbox(data_Y.Mean, lags=range(1,50), return_df=True,boxpierce=True)

p_values_McLeod_Li = sm.stats.acorr_ljungbox(data_Y.Mean**2, lags=range(1,50), return_df=True).lb_pvalue
p_values_Pierce = test_Q.lb_pvalue
p_values_Ljung = test_Q.bp_pvalue

fig = plt.figure(figsize=(8,4), dpi=300, tight_layout = True)
fig.suptitle("P-values of the Ljung-Box ,Box-Pierce test and McLeod-Li",y = 0.95
             ,size='x-large',weight = 'roman')


plt.plot(p_values_Pierce,'o-',markersize = 4,label='Box-Pierce')
plt.plot(p_values_Ljung,'-o',markersize = 4,label='Ljung-Box')
plt.plot(p_values_McLeod_Li,'-o',markersize = 4,label='McLeod-Li')
plt.semilogy()

fig.legend(loc = (0.82,0.7))
plt.xlabel('Autocorrelation lags')
plt.ylabel('p-values')

plt.savefig("figure/p_values_Ljung_Pierce_McLeod.png", dpi=300, bbox_inches='tight')
plt.close(fig)

# Definition generic function for plots

In [95]:
def plot_ljung_pierce(data, loca = (0.82,0.7), semilog_y = True,fname ='figure/p_values_Ljung_Pierce_McLeod_test.png', closefig = False,alpha = 0.05,df=0):
    test_Q = sm.stats.acorr_ljungbox(data, lags=range(1,50), return_df=True,boxpierce=True,model_df = df)

    p_values_McLeod_Li = sm.stats.acorr_ljungbox(data**2, lags=range(1,50), return_df=True,model_df = df).lb_pvalue
    p_values_Pierce = test_Q.lb_pvalue
    p_values_Ljung = test_Q.bp_pvalue
    
    fig = plt.figure(figsize=(8,4), dpi=300, tight_layout = True)
    fig.suptitle("P-values of the Ljung-Box ,Box-Pierce test and McLeod-Li",y = 0.95
                 ,size='x-large',weight = 'roman')


    plt.plot(p_values_Pierce,'o-',markersize = 4,label='Box-Pierce')
    plt.plot(p_values_Ljung,'-o',markersize = 4,label='Ljung-Box')
    plt.plot(p_values_McLeod_Li,'-o',markersize = 4,label='McLeod-Li')
    
    if (semilog_y): plt.semilogy()
    plt.axhline(y=alpha,linestyle='--',label='p_value = '+str(alpha))
    fig.legend(loc = loca)
    plt.xlabel('Autocorrelation lags')
    plt.ylabel('p-values')

    plt.savefig(fname, dpi=300, bbox_inches='tight')
    if (closefig) : plt.close(fig)
    return 

In [50]:
def plot_acf_pacf(data,lag = 50,closefig=False,fname = "figure/Annual_acf_pacf_test.png",title = 'ACF and PACF'):
    fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 300)
    
    fig.suptitle(title,y = 1.1,size='x-large',weight = 'roman')
    
    plot_acf(data, lags=lag,ax=axes[0], color='k',zero = False)
    plot_pacf(data, lags=lag,ax=axes[1], color='k',zero = False)
    
    axes[0].set(xlabel="lags")
    axes[1].set(xlabel="lags")
    
    
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    if (closefig): plt.close(fig)
    return

In [51]:
def plot_test_U_MannWhitney(data,alt = 'two-sided',alpha = 0.05, lag = 1,loca = (0.775,0.73),fname="figure/p_values_MannWhitneyTest_test.png",closefig=False):
    p_values = np.array([])
        
    for i in range(lag,np.shape(data)[0]-lag):
        x = data[:i]
        y = data[i:]
        p = np.array([sc.stats.mannwhitneyu(x, y, use_continuity=True, alternative=alt).pvalue])
        # p = np.array([sc.stats.kendalltau(x, y).pvalue])
        p_values = np.concatenate([p_values,p])

    
    fig = plt.figure(figsize=(8,4), dpi=300, tight_layout = True)
    fig.suptitle("P-values of the Mann-Whitney U test",y = 1.,size='x-large',weight = 'roman')
    
    line1, =plt.plot('o',color='red',label='p-value > '+str(alpha))
    line2, =plt.plot('o',color='grey',label='p-value <= '+str(alpha))
    
    for i in range(lag,np.shape(data)[0]-lag):
        if (p_values[i-1]>alpha):
            line1, = plt.plot(i,p_values[i-1],'o',color='red',label = 'p-value > '+str(alpha))
        else:
            line2, =plt.plot(i,p_values[i-1],'o',color='grey',label='p-value <= '+str(alpha))
    
        plt.axhline(y=alpha,linestyle='--',label='p_value = '+str(alpha))

    fig.legend([line1,line2],['p-value > '+str(alpha),'p-value <= '+str(alpha)],loc = loca)
    plt.xlabel('Size of \'left\' sample in the U test')
    plt.ylabel('p-values')
    
    
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    if (closefig): plt.close(fig)
    
    return

In [52]:
def qq_plot(data,fname = "figure/Annual_qqplot_test.png"):
    fig = qqplot(data,loc = data.mean(), scale = data.std(), line ='45')
    fig.suptitle("QQ-plot",y = 0.95,size='xx-large',weight = 'roman')
    plt.show()
    
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.close(fig)
    return

In [53]:
def ecdf_vs_cdf(data,fname ="figure/Annual_ecdf_cdf_test.png", closefig=False):
    ecdf = ECDF(data)
    mean = data.mean()
    std = data.std()
    x = np.linspace(-4.+mean,4.+mean,5000)
    
    fig, axs = plt.subplots(1,2, figsize=(14,4))
    
    axs[0].plot(x,ecdf(x),"k",label='ecdf')
    axs[0].plot(x,sc.stats.norm.cdf(x,loc = mean,scale = std), "b--", label = 'cdf')
    axs[1].plot(x,np.abs(ecdf(x)-sc.stats.norm.cdf(x,loc = mean ,scale = std)),color = 'k')
    
    axs[0].set(xlabel="x",ylabel="P (X<=x)")
    axs[1].set(xlabel="x",ylabel="Absolute deviation")
    axs[0].set_title("Ecdf vs cdf")
    axs[1].set_title("Absolute deviation of the ecdf from the cdf")
    
    axs[0].legend(loc = (0.05,0.8))
    
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    if (closefig): plt.close(fig)
    return

# Model Identification and Selection

In [54]:
break_year = 1962

mean1 = data_Y.Mean[data_Y.Years<break_year].mean()
data1 = data_Y[data_Y.Years<break_year].copy()

mean2 = data_Y.Mean[data_Y.Years>=break_year].mean()
data2 = data_Y[data_Y.Years>=break_year].copy()

n = np.shape(data_Y.Mean)[0]
t = np.array(data_Y.Years).reshape((n,1))
one = np.ones(shape=(np.shape(t)[0],1))
X = np.concatenate([one,t],axis = 1)

n1 = np.shape(data1.Mean)[0]
n2 = np.shape(data2.Mean)[0]
t1 = np.array(data1.Years).reshape((n1,1))
t2 = np.array(data2.Years).reshape((n2,1))
one1 = np.ones(shape=(np.shape(t1)[0],1))
X1 = np.concatenate([one1,t1],axis = 1)
one2 = np.ones(shape=(np.shape(t2)[0],1))
X2 = np.concatenate([one2,t2],axis = 1)

acf1 = acf(data1.Mean,nlags=np.shape(data1.Mean)[0],fft=False)
sigma1 = data1.Mean.var()*sc.linalg.toeplitz(acf1)
GLS_reg1 = sm.GLS(data1.Mean,X1,sigma1).fit()

acf2 = acf(data2.Mean,nlags=np.shape(data2.Mean)[0],fft=False)
sigma2 = data2.Mean.var()*sc.linalg.toeplitz(acf2)
GLS_reg2 = sm.GLS(data2.Mean,X2,sigma2).fit()

acf_ = acf(data_Y.Mean,nlags=np.shape(data_Y.Mean)[0],fft=False)
sigma_ = data_Y.Mean.var()*sc.linalg.toeplitz(acf_)
GLS_reg = sm.GLS(data_Y.Mean,X,sigma_).fit()

coef1 = GLS_reg1.params
coef2 = GLS_reg2.params
coef = GLS_reg.params

f = lambda x: coef[0]+coef[1]*x
f1 = lambda x: coef1[0]+coef1[1]*x
f2 = lambda x: coef2[0]+coef2[1]*x
f_split = lambda x: (x<break_year)*f1(x)+(x>=break_year)*f2(x)

In [55]:
#On sauvegarde les summary pour les deux régréssion
summa1 = GLS_reg1.summary()
summa2 = GLS_reg2.summary()
summa = GLS_reg.summary()

# Holm-Bonferroni's Procedure avec alpha = 0.1 pour joindre les deux tests 

g = open('txt/summary1901-1962_GLS.txt','w')
g.write(summa1.as_latex())
g.close()
g = open('txt/summary1962-2021_GLS.txt','w')
g.write(summa2.as_latex())
g.close()
g = open('txt/summary_GLS.txt','w')
g.write(summa.as_latex())
g.close()

In [56]:
#Plot split-regression
fig = plt.figure(figsize=(5,3), dpi=300, tight_layout = True)
fig.suptitle("Modelisation of the trend by GLS on two succesive period",y = 0.95,size='x-large',weight = 'roman')

plt.plot(data1.Years,f_split(data1.Years),'r',label='1901-1961 GLS fit')
plt.plot(data2.Years,f_split(data2.Years),'b',label='1962-2021 GLS fit')
plt.plot(data_Y.Mean,'grey')

plt.xlabel('Year')
plt.ylabel('Temperature (°C)')
fig.legend(['1901-1961 GLS fit','1962-2021 GLS fit'],loc = (0.15,0.67))

plt.savefig('figure/GLS_split_period', dpi=300, bbox_inches='tight')
plt.close()

In [57]:
#Plot entiere regression
fig = plt.figure(figsize=(5,3), dpi=300, tight_layout = True)
fig.suptitle("Modelisation of the trend by GLS",y = 0.95,size='x-large',weight = 'roman')

plt.plot(Years,f(Years),label = '1901-2021 GLS fit')
plt.plot(data_Y.Mean,'grey')

plt.xlabel('Year')
plt.ylabel('Temperature (°C)')
fig.legend(['1901-2021 GLS fit'],loc = (0.15,0.74))

plt.savefig('figure/GLS_full_period', dpi=300, bbox_inches='tight')
plt.close()

In [58]:
#On fixe la tendance comme étant la tendance calculé sur deux période
data_Y["Mean_detrended"] = data_Y.Mean-f_split(Years)

In [59]:
fig = plt.figure(figsize=(5,3), dpi=300, tight_layout = True)
fig.suptitle("Detrended mean temperature",y = 0.91,size='x-large',weight = 'roman')

plt.plot(data_Y.Mean_detrended,'grey')

plt.xlabel('Years')
plt.ylabel('°C')

plt.savefig('figure/detrended_mean_temperature', dpi=300, bbox_inches='tight')
plt.close()

In [60]:
plot_ljung_pierce(data_Y.Mean_detrended,semilog_y = False,loca = (0.78,0.66),closefig=True
                 ,fname = 'figure/Detrended_mean_Ljung_Pierce.png')
plot_acf_pacf(data_Y.Mean_detrended,lag=50, title = 'ACF and PACF of annual detrended mean temperature'
              ,closefig=True,fname = 'figure/Detrended_mean_acf_pacf.png')

In [61]:
#Stationarity test on detrended mean with addfuller
test_addfuller = adfuller(data_Y.Mean_detrended, maxlag=None, regression='nc', autolag='AIC'
                          , store=False, regresults=True)

g = open('txt/addfullerTest_annual.txt','a')
g.write("Annual detrended mean (no constant) : p-value = "+str(test_addfuller[1])+'\n')
g.close()

In [183]:
#Model selection using AIC
p = np.array(np.arange(0,2))
q = np.array(np.arange(0,3))

param = np.array([np.array([int(pp),int(qq)]) for pp in p for qq in q])

AIC = np.zeros(shape=(np.shape(param)[0],))

model_selection = pd.DataFrame(np.array([param[:,0],param[:,1],AIC]).T, columns = ["param_p","param_q","AIC"])

for i, (pp,qq) in enumerate(param):
    arma_mod = ARIMA(data_Y.Mean_detrended, order=(pp,0,qq)).fit()
    model_selection.AIC[i] = arma_mod.aicc

i = np.argmin(model_selection.AIC)
mod = model_selection.to_latex()

g = open('txt/ModelSelection_annual.txt','w')
g.write(mod)
g.close()


In [156]:


arma_mod = ARIMA(data_Y.Mean_detrended, order=(0,0,2)).fit()
print(arma_mod.resid)


#print(arma_mod.aic)


#plot_ljung_pierce(res,semilog_y = False,loca = (0.78,0.66),df = 4)
#plot_acf_pacf(res,lag=50, title = 'ACF and PACF of annual detrended mean temperature')



0     -0.501545
1     -0.236140
2     -0.072053
3      0.567586
4     -0.260337
         ...   
116   -0.045760
117    1.047590
118    0.284542
119    0.460370
120   -0.698351
Length: 121, dtype: float64


0     0.493401
1    -0.059284
2     0.231543
3     0.071956
4    -0.009428
        ...   
95   -0.190227
96    0.200075
97    0.046830
98    0.591670
99   -0.368304
Name: Mean_detrended, Length: 100, dtype: float64

In [ ]:
arma_mod = ARIMA(data_Y.Mean_detrended, order=(1,0,5),enforce_stationarity = True, enforce_invertibility = True).fit()
dif = arma_mod.resid

plot_ljung_pierce(dif,semilog_y = False)
plot_acf_pacf(dif,lag=28)
qq_plot(dif)
ecdf_vs_cdf(dif)
plot_test_U_MannWhitney(dif,alt = 'two-sided',loca = (0.77,0.87))